This markdown file takes the raw reads for all individuals from each population (these can be found on NCBI), filters them (based on quality, etc.), aligns them to a reference transcriptome, and then counts the number of reads mapping to each transcript per sample.

The output is a counts matrix, which is subsequently used for all differential transcript analysis.

This code follows along using this pipeline: https://github.com/z0on/tag-based_RNAseq/blob/master/tagSeq_processing_README.txt and these scripts: https://github.com/z0on/tag-based_RNAseq. 

In [2]:
%pwd

'/home/mbitter/Projects/CG_pH'

FastQC of Raw Reads

In [ ]:
%%sh
#Load modules
module load java-jdk/1.8.0_92
module load fastqc/0.11.5

mkdir /scratch/mbitter/CGpH_scratch/FastQC_output/
fastqc -o /scratch/mbitter/CGpH_scratch/FastQC_output/ --extract /group/pfister-lab/Bitter/CG_pH/RawQauntSeqReads_20180529/*.fastq  -t 8

View output of FastQC using MultiQC. To view the output .html file, I simply downloaded it to my local computer and opened the file in a browser.

In [4]:
%%sh
#Load modules
module load gcc/6.2.0
module load python/2.7.13


multiqc -n raw_QuantSeq_CGpH_multiqc /scratch/mbitter/CGpH_scratch/FastQC_output/ -o  /scratch/mbitter/CGpH_scratch/Multiqc_output



Searching 840 files..


[WARNING]         multiqc : MultiQC Version v1.5 now available!
[INFO   ]         multiqc : This is MultiQC v1.3
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching '/scratch/mbitter/CGpH_scratch/FastQC_output/'
[INFO   ]          fastqc : Found 42 reports
[INFO   ]         multiqc : Compressing plot data
[WARNING]         multiqc : Previous MultiQC output found! Adjusting filenames..
[WARNING]         multiqc : Use -f or --force to overwrite existing reports instead
[INFO   ]         multiqc : Report      : ../../../../scratch/mbitter/CGpH_scratch/Multiqc_output/raw_QuantSeq_CGpH_multiqc_1.html
[INFO   ]         multiqc : Data        : ../../../../scratch/mbitter/CGpH_scratch/Multiqc_output/raw_QuantSeq_CGpH_multiqc_data_1
[INFO   ]         multiqc : MultiQC complete


QC areas of concern are: Per-base Sequence Content (all failed), Per-base GC content (41 failed), Sequence Duplication Levels (all failed), and Overrepresented Sequences (all have wanrings). 

Trimming the Reads

In [ ]:
%%sh
#Load modules
module load gcc/6.2.0
module load fastx_toolkit/0.0.14

#Make new directory for trimmed reads
mkdir /scratch/mbitter/CGpH_scratch/TrimmedReads

#Trim the first 10 bp's, polyA tails, adapters, and low quality reads (<Q20), and retain reads > 20 bp
for i in /group/pfister-lab/Bitter/CG_pH/RawQauntSeqReads_20180529/*.fastq;
    do fastx_trimmer -i $i -f 10 | \
    fastx_clipper -a AAAAAAAA -l 20 -Q33 | \
    fastx_clipper -a AGATCGGAAG -l 20 -Q33 | \
    fastq_quality_filter -Q33 -q 20 -p 90 > ${i/L008_R1_001.fastq/trim_10.fq};
done

#Move trimmed reads to trimmed directory
mv /group/pfister-lab/Bitter/CG_pH/RawQauntSeqReads_20180529/*trim_10* /scratch/mbitter/CGpH_scratch/TrimmedReads/





Re-run FastQC on Trimmed Files and once again use MultiQC to view output

In [ ]:
%%sh
module load java-jdk/1.8.0_92
module load fastqc/0.11.5

#Run FastQC
fastqc -o /scratch/mbitter/CGpH_scratch/FastQC_output/TrimmedReads_FastQC/ --extract /scratch/mbitter/CGpH_scratch/TrimmedReads/*fq -t 80


In [8]:
%%sh
#Load modules
module load gcc/6.2.0
module load python/2.7.13

multiqc -n trimmed_QuantSeq_CGpH_multiqc /scratch/mbitter/CGpH_scratch/FastQC_output/TrimmedReads_FastQC/ -o  /scratch/mbitter/CGpH_scratch/Multiqc_output//



Searching 840 files..


[WARNING]         multiqc : MultiQC Version v1.5 now available!
[INFO   ]         multiqc : This is MultiQC v1.3
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching '/scratch/mbitter/CGpH_scratch/FastQC_output/TrimmedReads_FastQC/'
[INFO   ]          fastqc : Found 42 reports
[INFO   ]         multiqc : Compressing plot data
[INFO   ]         multiqc : Report      : ../../../../scratch/mbitter/CGpH_scratch/Multiqc_output/trimmed_QuantSeq_CGpH_multiqc.html
[INFO   ]         multiqc : Data        : ../../../../scratch/mbitter/CGpH_scratch/Multiqc_output/trimmed_QuantSeq_CGpH_multiqc_data
[INFO   ]         multiqc : MultiQC complete


Looks much better for per-base sequence content and per-base GC content . It looks worse for sequence duplication levels (in all sequences), and overrepresented sequences (in 37 samples)

Mapping reads to reference transcriptome that eliminates all putative isoforms (Moreira et al. 2015) using BowTie2 

In [ ]:
%%sh
#Load modules
module load gcc intel
module load bowtie2

#Build index using Moreira 2015 M. galloprovincialis transcriptome.
bowtie2-build --threads 8 -f /group/pfister-lab/Bitter/PublishedSequences/MoreiraTranscriptome/Moreira2015_Mgallo_transcriptome.fa /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MgalloIndex

cd /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/

#Run Bowtie on one practice sample
bowtie2 --local -x /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MgalloIndex -U /scratch/mbitter/CGpH_scratch/TrimmedReads/CP-KS-MB-Lib1-CGpH-13_S14_trim_10.fq -S $S --no-hd --no-sq --no-unal -k 5 -p 8 2 

In [2]:
%%sh

#Loop through all files and run bowtie
S="/scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/${i/trim_10.fq/bt2.sam}"
out="/scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/${i/trim_10.fq/bt2.stout}"
bowtie2 --local -x /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MgalloIndex -U /scratch/mbitter/CGpH_scratch/TrimmedReads/CP-KS-MB-Lib1-CGpH-13_S14_trim_10.fq -S PracticeAlignment.sam
#Load modules
module load gcc intel
module load bowtie2
#Map Trimmed reads against Index
cd /scratch/mbitter/CGpH_scratch/TrimmedReads
for i in *.fq; do
    S="/scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/${i/trim_10.fq/bt2.sam}"
    out="/scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/${i/trim_10.fq/bt2.stout}"
    bowtie2 --local -x /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MgalloIndex -U $i -S $S --no-hd --no-sq --no-unal -k 5 -p 8 2> $out;
done



Lmod is automatically replacing "gcc/6.2.0" with "intel/2017"



Now let's check the output from an individual alignment

In [1]:
%%sh
cat /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/CP-KS-MB-Lib1-CGpH-28_S31_bt2.stout


4835124 reads; of these:
  4835124 (100.00%) were unpaired; of these:
    1179960 (24.40%) aligned 0 times
    1540365 (31.86%) aligned exactly 1 time
    2114799 (43.74%) aligned >1 times
75.60% overall alignment rate


Now, let's use python to print out summaries for each sample

In [ ]:
import glob, os
files = []
os.chdir("/scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/")
for file in glob.glob("*.stout"):
    files.append(file)

reads = []
maps= []
multiple = []
for f in files:
    name = '-'.join(f.split('_')[0].split('-')[2:8])
    print(name)
    IN = open(f,"r")
    read = IN.readline().split()[0]
    read = int(read)
    reads.append(read)
    print("Reads: "+str(read))
    IN.readline()
    IN.readline()
    IN.readline()
    mult = float(IN.readline().split()[1][1:-2])
    multiple.append(mult)
    print("Multiple: "+str(mult))
    mapping = float(IN.readline().split()[0][:-1])
    maps.append(mapping)
    print("Overall mapping: "+str(mapping))

print("Mean reads: "+str(sum(reads)/float(len(reads))))
print("Mean mapping multiple: "+str(sum(multiple)/float(len(multiple))))
print("Mean mapping: "+str(sum(maps)/float(len(maps))))
print("Number of samples: "+str(len(reads)))

In [ ]:
import glob, os
files = []
os.chdir("/scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/")
for file in glob.glob("*.stout"):
    files.append(file)

reads = []
maps= []
multiple = []
for f in files:
    name = '-'.join(f.split('_')[0].split('-')[2:8])
    print(name)


Generating counts matrix - Count the number of transcripts mapping to each gene/isoform from the transcriptome. 

In [1]:
#Generating table with 'reference_seq<tab>gene_ID' that will be used to identify read counts of aligned reads.
import glob, os
import re
import pyfaidx
import pandas as pd


os.chdir("/scratch/mbitter/CGpH_scratch/")


ReferenceTranscriptome = pyfaidx.Fasta("./ReferenceTranscriptome.fasta")
Gene_List = list(ReferenceTranscriptome.keys())


gene_dict = {}
cnt = 1
for i in Gene_List:
    cur_gene = i.split('.')[0]
    if cur_gene not in gene_dict:
        gene_dict[cur_gene] = cnt
        cnt +=1
final_dict = {}      
for i in Gene_List:
    cur_id = i.split('.')[0]
    final_dict[i] = gene_dict[cur_id]


Isos = list(final_dict.keys())
Genes = list(final_dict.values())

df = pd.DataFrame({"ref_seq": Isos})
df["gene_id"] = Genes
df.head()

df.to_csv("./gene_ids.tsv", sep="\t", index=False, header=False)




Using samcount.pl to count number of reads mapping to each gene/isoform.

First, practice using one file.

In [3]:
%%sh

cd /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/

/scratch/mbitter/CGpH_scratch/tag-based_RNAseq/samcount.pl ./CP-KS-MB-Lib1-CGpH-12_S37_bt2.sam /scratch/mbitter/CGpH_scratch/gene_ids.tsv aligner=bowtie2 > CP-KS-MB-Lib1-CGpH-12_S37_bt2.sam.count


disregarding reads mapping to multiple isogroups


It works! Now, loop through all libraries/mapped reads.

In [ ]:
%%sh

cd /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/

for i in *.sam; do
    /scratch/mbitter/CGpH_scratch/tag-based_RNAseq/samcount.pl $i /scratch/mbitter/CGpH_scratch/gene_ids.tsv aligner=bowtie2 >${i/.sam/.sam.counts}
done
    

Now, generate the counts.txt (matrix) 

In [6]:
%%sh

cd /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/

/scratch/mbitter/CGpH_scratch/tag-based_RNAseq/expression_compiler.pl *.sam.counts > EviOkAlt_allcounts.txt

In [8]:
%%sh
head -n 1 /scratch/mbitter/CGpH_scratch/TranscriptomeAlignment/MappedReads/EviOkAlt_allcounts.txt

	CP-KS-MB-Lib1-CGpH-10_S39_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-11_S28_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-12_S37_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-13_S14_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-14_S6_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-15_S20_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-16_S33_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-17_S2_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-18_S21_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-19_S19_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-1_S13_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-20_S3_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-21_S17_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-22_S18_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-22rep_S41_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-23_S30_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-24_S32_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-25_S5_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-26_S23_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-27_S22_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-28_S31_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-29_S24_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-2_S15_bt2.sam.counts	CP-KS-MB-Lib1-CGpH-30_S29_bt2.sam.counts	CP-KS-MB-Lib1-CGpH

After completing the above, I moved all data to the /group/pfister-lab/ and /home/mbitter directories.

I also re-named the EviAllCounts file to CGpH_Counts.csv, and it is now located in /home/mbitter/CG_pH/Data

In [40]:
import glob, os
import pandas as pd

os.chdir("/home/mbitter/Projects/CG_pH/Data/")

data = pd.read_csv("CGpH_allcounts.txt", sep = "\t")
# Remove Unnamed 43
data = data.drop('Unnamed: 43', axis=1)


# re-assigning columns
cur_colnames = data.columns
new_colnames = []
for c in cur_colnames:
    if "Unnamed" not in c:
        col_split = c.split('-')
        cgph = col_split[4]
        individual = col_split[5].split('_')[0]
#         new_colnames.append("%s.%s" % (cgph,individual))
        new_colnames.append(cgph + "." + individual)
    else:
        new_colnames.append("Gene")

# Re-assign column names
data.columns = new_colnames


data.to_csv("./CGpH_Counts.csv", sep=",", header=True, index=False)
data.shape


(101209, 43)